# Dependencies

In [ ]:
!pip install yargy
!pip install dateparser
!pip install rutimeparser
!pip install spacy-stanza

In [ ]:
import stanza
stanza.download('ru') 

# Exctract time expressions

In [1]:
import re

with open("DeepPavlov_7777.txt", "r") as f:
    text = f.read()

sentences = list()
sent_ids = list()
sentences_parsed = text.split("\n\n")
for sent in range(len(sentences_parsed)-1):
    pattern = "# sent_id = (.*?)\n"
    sent_ids.append(re.search(pattern, sentences_parsed[sent]).group()[12:-1])
    sentences_parsed[sent] = re.sub(pattern, '', sentences_parsed[sent])
    pattern = "# text = (.*?)\n"
    sentences.append(re.search(pattern, sentences_parsed[sent]).group()[9:-1])
    sentences_parsed[sent] = re.sub(pattern, '', sentences_parsed[sent])

In [296]:
%%time
import time_expressions

time_exp = time_expressions.extract(sentences)

CPU times: user 4min 5s, sys: 98.6 ms, total: 4min 6s
Wall time: 4min 6s


In [432]:
time_exp[:10]

[['в течении 5 лет'],
 ['с 1993 года'],
 ['В мае 2009 года'],
 ['в конце 2009 , начале 2010 года'],
 ['В апреле 2010 года', 'за сутки'],
 ['В течение месяца'],
 ['около 6 лет', 'впервые'],
 ['В 2008 году'],
 ['17.09.2008'],
 ['Осенью 2010 года']]

# Normalize time expressions

In [ ]:
%%time
norm_time_exp = time_expressions.normalize(time_exp)

In [445]:
import pandas as pd

d = {'id': sent_ids, 'sentence': sentences, 'time_expressions': time_exp, 'norm_time_expressions': norm_time_exp}
df = pd.DataFrame(data=d)
df.to_csv('result_with_norm.csv', index = False)
df.head(10)

,id,sentence,time_expressions,norm_time_expressions
0,0,Болеет СД 2 типа в течении 5 лет.,[в течении 5 лет],[[None]]
1,1,"Считает себя больной с 1993 года, когда перене...",[с 1993 года],[1993-08-01 00:00:00]
2,2,В мае 2009 года в СибГМУ проводилась РЧ-аблаци...,[В мае 2009 года],[2009-05-01 00:00:00]
3,3,"Ухудшение в конце 2009, начале 2010 года, когд...","[в конце 2009 , начале 2010 года]","[[2009-12-01 00:00:00, 2010-01-01 00:00:00]]"
4,4,В апреле 2010 года проходила лечение в отд НРС...,"[В апреле 2010 года, за сутки]","[2010-04-01 00:00:00, None]"
5,5,В течение месяца чувствовала себя удовлетворит...,[В течение месяца],[None]
6,6,"Больной себя считает около 6 лет, когла вперв...","[около 6 лет, впервые]","[[None], None]"
7,7,"В 2008 году внезапно почувствовала слабость, п...",[В 2008 году],[2008-08-01 00:00:00]
8,8,"17.09.2008 пациентке имплантирован ЭКС 452, па...",[17.09.2008],[2008-09-17 00:00:00]
9,9,Осенью 2010 года пациентка госпитализировалась...,[Осенью 2010 года],[[2010-09-01 00:00:00]]


In [431]:
norm_time_exp[:10]

[[None],
 [datetime.datetime(1993, 8, 1, 0, 0)],
 [datetime.datetime(2009, 5, 1, 0, 0)],
 [[datetime.datetime(2009, 12, 1, 0, 0), datetime.datetime(2010, 1, 1, 0, 0)]],
 [datetime.datetime(2010, 4, 1, 0, 0), None],
 [None],
 [datetime.datetime(2020, 6, 1, 0, 0), None],
 [datetime.datetime(2008, 8, 1, 0, 0)],
 [datetime.datetime(2008, 9, 17, 0, 0)],
 [[datetime.datetime(2010, 9, 1, 0, 0)]]]